In [ ]:
%cd "D:\Users\sean.ogara\Documents\ons-energy-analysis"

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

from src.visualisations.scatter_chart import scatter_chart

In [ ]:
electricity_price_data = pd.read_csv("data/processed/global_electricity_household_prices.csv")
electricity_price_data.head()

In [ ]:
#distribution of prices
num_bins = 10
n, bins, patches = plt.hist(electricity_price_data['price'], num_bins,
                            density = 1,
                            color ='green',
                            alpha = 0.7)

plt.xlabel('Price (£/kWh)')
plt.ylabel('Count')
plt.title('International Electricty Prices December 2022',
          fontweight = 'bold')
plt.show()

In [ ]:
#global mean price
global_mean_price = electricity_price_data['price'].mean()
global_mean_price

In [ ]:
#uk price
uk_price = electricity_price_data[electricity_price_data['iso2'] == 'GB']['price'].values[0]
uk_price

In [ ]:
#ratio between uk and global mean
uk_price / global_mean_price

In [ ]:
#bring in energy mix data
energy_mix_data = pd.read_csv("data/raw/owid-energy-data.csv")
energy_mix_data.tail()

In [ ]:
#filter for 2021 as this is the most recent complete data
energy_mix_2021 = energy_mix_data.loc[energy_mix_data['year'] == 2021, :]
energy_mix_2021.head()

In [ ]:
#fileter for columns of interest
cols_of_interest = [
                    'country', 
                    'year',
                    'iso_code',
                    'population',
                    'energy_per_capita',
                    'fossil_elec_per_capita',
                    'low_carbon_energy_per_capita',
                    'low_carbon_share_energy',
                    'low_carbon_share_elec',
                    'gas_prod_per_capita',
                    'oil_prod_per_capita',
                    'gas_production',
                    'oil_production'
                    ]           
energy_mix_2021 = energy_mix_2021[cols_of_interest]

In [ ]:
energy_mix_2021['energy_per_capita'].hist()

In [ ]:
energy_mix_2021['energy_per_capita_log'] = np.log10(energy_mix_2021[energy_mix_2021['energy_per_capita'] != 0]['energy_per_capita'])
energy_mix_2021['energy_per_capita_log'].hist()

In [ ]:
energy_mix_2021['gas_prod_per_capita'].hist()

In [ ]:
energy_mix_2021['oil_prod_per_capita'].hist()

In [ ]:
energy_mix_2021.describe()

In [ ]:
energy_mix_2021[['oil_prod_per_capita', 'gas_prod_per_capita']] = energy_mix_2021[['oil_prod_per_capita', 'gas_prod_per_capita']].fillna(0)


In [ ]:
energy_mix_2021.describe()

In [ ]:
#combine oil and gas energy production per cap for total and log to make normally distributed
energy_mix_2021['oil_and_gas_prod_per_capita'] = energy_mix_2021['oil_prod_per_capita'] + energy_mix_2021['gas_prod_per_capita']
#fill all 0 values as 1 before log
energy_mix_2021['oil_and_gas_prod_per_capita'].replace(0, np.nan, inplace=True)
energy_mix_2021['oil_and_gas_prod_per_capita_log'] = np.log10(energy_mix_2021['oil_and_gas_prod_per_capita'])
energy_mix_2021['oil_and_gas_prod_per_capita_log'].hist()

In [ ]:
energy_mix_2021['low_carbon_energy_per_capita'].hist()

In [ ]:
energy_mix_2021['low_carbon_energy_per_capita_log'] = np.log10(energy_mix_2021['low_carbon_energy_per_capita'])
energy_mix_2021['low_carbon_energy_per_capita_log'].hist()

In [ ]:
#join on 2018 gdp data
joined_energy_data = pd.merge(left=energy_mix_data.loc[energy_mix_data['year'] == 2018, :][['iso_code', 'gdp']], right=energy_mix_2021, on='iso_code')

In [ ]:
#join with price data
joined_energy_data = pd.merge(left=electricity_price_data[['iso3', 'price']], right=joined_energy_data, left_on='iso3', right_on='iso_code')

In [ ]:
#check data
joined_energy_data.describe()

In [ ]:
joined_energy_data['gdp_per_cap'] = joined_energy_data['gdp'] / joined_energy_data['population']
joined_energy_data['gdp_per_cap_log'] = np.log10(joined_energy_data['gdp'] / joined_energy_data['population'])

In [ ]:
scatter_chart(data=joined_energy_data, x_var='gdp_per_cap_log', y_var='price', x_label='GDP log($)', y_label='Price (£/kWh)', hover_labels='country')

In [ ]:
scatter_chart(data=joined_energy_data, x_var='oil_and_gas_prod_per_capita_log', y_var='price', x_label='Oil and gas production per capita log(tWH)', y_label='Price (£/kWh)', hover_labels='country')

In [ ]:
#global relationship of price against renewable share shows negative correlation, this could be caused by higher standards of leaving in countries with high renewable share
scatter_chart(data=joined_energy_data, x_var='low_carbon_share_elec', y_var='price', x_label='Low carbon electricity share (%)', y_label='Price (£/kWh)', hover_labels='country')

In [ ]:
#eu + uk only

europe_country_codes = [
    'GBR', 'AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 
    'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 
    'SVK', 'SVN', 'ESP', 'SWE'
]

#opposite trend is true for europe but weak relationship
scatter_chart(data=joined_energy_data[joined_energy_data['iso3'].isin(europe_country_codes)], x_var='low_carbon_share_elec', y_var='price', x_label='Low carbon electricity share (%)', y_label='Price (£/kWh)', hover_labels='country')

In [ ]:
#plot of price vs usage per cap, again this could be driven by higher standards of living driving up the prices in countries which use more energy
scatter_chart(data=joined_energy_data, x_var='energy_per_capita_log', y_var='price', x_label='Energy usage per capita log(tWH)', y_label='Price (£/kWh)', hover_labels='country')

In [ ]:
#remove energy per cap as it correlates with gdp per cap
features = joined_energy_data[['price', 'gdp_per_cap_log', 'oil_and_gas_prod_per_capita_log', 'low_carbon_share_elec', 'energy_per_capita_log']]
corr_matrix = features.corr()


f, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_matrix, annot=True, mask = mask, cmap=cmap)

In [ ]:
#drop target
features.drop(['price', 'energy_per_capita_log'], axis=1, inplace=True)

#add constant to predictor variables
features = sm.add_constant(features)

#assign target var
target = joined_energy_data[['price']]

#fit linear regression model
model = sm.OLS(target, features, missing='drop').fit()

#view model summary
print(model.summary())

In [ ]:
#scaled model to get feature importance
scaler = StandardScaler()

features = joined_energy_data[['gdp_per_cap_log', 'oil_and_gas_prod_per_capita_log', 'low_carbon_share_elec']]

features_scaled = scaler.fit_transform(features)
target_scaled = scaler.fit_transform(target)

In [ ]:
#make regression model to explain price - does renewable mix matter? Nope, not once you've controlled for gdp per cap.

features_scaled = sm.add_constant(features_scaled)

#fit linear regression model
scaled_model = sm.OLS(target_scaled, features_scaled, missing='drop').fit()

#view model summary
print(scaled_model.summary())


In [ ]:
#create data for clustering
cluster_data = joined_energy_data[['price', 'gdp_per_cap_log', 'oil_and_gas_prod_per_capita_log', 'low_carbon_share_elec']].dropna().values

In [ ]:
from sklearn.cluster import KMeans

#find the optimal number of clusters using elbow method
#using Within-Cluster Sum of Square as variance metric - WCSS
WCSS = []
for i in range(1,11):
    model = KMeans(n_clusters= i, init="random", # method for selecting initial cluster points
                      max_iter=10, # iterations before stopping
                      random_state=123)
    model.fit(cluster_data)
    WCSS.append(model.inertia_)
fig = plt.figure(figsize = (7,7))
plt.plot(range(1,11), WCSS, linewidth=4, markersize=12, marker='o',color = 'green')
plt.xticks(np.arange(11))
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")
plt.show()

In [ ]:
#k=4 is optimal
x=cluster_data
model = KMeans(n_clusters = 4, init = 'random', max_iter = 10)
y_clusters = model.fit_predict(cluster_data)



In [ ]:
y_clusters

In [ ]:
x[y_clusters == 0]

In [ ]:
#2D plot

plt.figure(figsize = (20,10))
plt.scatter(x[y_clusters == 0,1],x[y_clusters == 0,0],s = 50, c = 'green', label = "1")
plt.scatter(x[y_clusters == 1,1],x[y_clusters == 1,0],s = 50, c = 'blue', label = "2")
plt.scatter(x[y_clusters == 2,1],x[y_clusters == 2,0],s = 50, c = 'black', label = "3")
plt.scatter(x[y_clusters == 3,1],x[y_clusters == 3,0],s = 50, c = 'red', label = "4")
plt.scatter(model.cluster_centers_[:,1],model.cluster_centers_[:,0], s = 100, c = "yellow", label = "centroids")
plt.xlabel("Price (£/kWh")
plt.ylabel("GDP per cap log($)")
plt.legend()
plt.show()

In [ ]:
# display 3 var results using 3d scatter (renewable share, oil + gas prod, price)
#display 4 var results on 2d plot using pca
#get avg values for each cluster and find which cluster uk is in
#could impute values to make cluster results better
#use regression model to see if UK is outlier for elec price based on input var